In [1]:
import pandas as pd, gc

# Assigning to data which will be used for training regression model

In [4]:
df_train_regress = pd.read_csv('df_train_regress.csv',dtype={'Customer ID':'object'})
df_train_regress['Date'] = pd.to_datetime(df_train_regress['Date'])

In [5]:
cc_dtype = {'Customer ID':'object',
            'Customer_Cluster ':'int8'}

In [6]:
df_customer_cluster = pd.read_csv('customer_cluster_train.csv', dtype =cc_dtype)

In [7]:
df_customer_cluster = df_customer_cluster[['Customer ID','Customer_Cluster']]

In [8]:
gc.collect()

80

In [9]:
df_train_regress = pd.merge(df_train_regress,df_customer_cluster,how='left',on=['Customer ID'])

In [10]:
df_train_regress.to_csv('df_train_regress_cc.csv',index=False)

# Assigning Customer_Cluster to test data with following logic  
1. if customer id in any of the cluster we got from training data, then that Customer_Cluster,  

2. if it is a customer which was visited by ABC representative but never ordered, then Customer_Cluster = 0. Quantity estimate for these customers will be zero as they may not be buying because of some external factor. This also helps us manage the huge number of customers.

3. if new customer then assigned Customer_Cluster = 1 as it has low visits and makes most business sense  

In [13]:
df_test = pd.read_csv('df_test.csv',dtype = {'Customer ID':'object'})

In [14]:
df_test.shape

(842484, 19)

In [15]:
df_test.head()

,Customer ID,Quantity,Invoice Value,Date,month,year,dayofmonth_sin,dayofmonth_cos,month_sin,month_cos,week_sin,week_cos,dayofweek_sin,dayofweek_cos,quarter_sin,quarter_cos,month_start_mid_end_sin,month_start_mid_end_cos,price
0,500072277,1.0,45.62,2019-03-23,3,2019,-0.9985,-0.05066,1.0,0.0,0.9927,0.12054,-0.9750,-0.2225,1.0,0.0,-0.000,1.0,45.62500
1,500105538,10.0,416.50,2019-03-31,3,2019,-0.0000,1.00000,1.0,0.0,1.0000,0.00000,-0.7817,0.6235,1.0,0.0,-0.000,1.0,41.65625
2,500677199,0.0,0.00,2019-03-17,3,2019,-0.2993,-0.95400,1.0,0.0,0.9707,0.23930,-0.7817,0.6235,1.0,0.0,-0.866,-0.5,44.43750
3,500713833,0.0,0.00,2019-03-20,3,2019,-0.7910,-0.61230,1.0,0.0,0.9927,0.12054,0.9750,-0.2225,1.0,0.0,-0.866,-0.5,44.43750
4,500033437,0.0,0.00,2019-03-22,3,2019,-0.9683,-0.25070,1.0,0.0,0.9927,0.12054,-0.4338,-0.9010,1.0,0.0,-0.000,1.0,44.43750


1. if customer id in any of the cluster we got from training data, then that Customer_Cluster,  

In [16]:
df_merge = pd.merge(df_test,df_customer_cluster, how='left', on=['Customer ID'])

In [17]:
df_test_1 = df_merge[~df_merge['Customer_Cluster'].isnull()]

In [18]:
df_nan_1 = df_merge[df_merge['Customer_Cluster'].isnull()]

In [19]:
df_nan_1.shape

(17827, 20)

2. if it is a customer which was visited by ABC representative but never ordered, then Customer_Cluster = 0. Quantity estimate for these customers will be zero as they may not be buying because of some external factor. This also helps us manage the huge number of customers.

In [20]:
df_train_zero_qty_customers = pd.read_csv('df_train_zero_qty_customers.csv', dtype = {'Customer ID':'object','Quantity':'int8'})

In [21]:
df_train_zero_qty_customers = df_train_zero_qty_customers['Customer ID']

In [22]:
df_train_zero_qty_customers.unique()

array(['0', '500000001', '500000009', ..., '899140257', '899140268',
       '899140269'], dtype=object)

In [23]:
gc.collect()

160

In [24]:
df_test_2 = df_nan_1[df_nan_1['Customer_Cluster'].isin(df_train_zero_qty_customers.unique())]

In [25]:
df_test_2['Customer_Cluster'] = 0

### 2019, ABC Sales representative did not visit the customers, who did not buy from them in 2018

In [26]:
df_test_2.shape

(0, 20)

3. if new customer then assigned Customer_Cluster = 1 as it has low visits and makes most business sense  

In [27]:
df_test_3 = df_nan_1[~df_nan_1['Customer_Cluster'].isin(df_train_zero_qty_customers.unique())]

In [28]:
df_test_3['Customer_Cluster'] = 1

In [29]:
df_test_1.shape[0]+df_test_2.shape[0]+df_test_3.shape[0]

842484

In [30]:
df_test_cc = df_test_1.append(df_test_2).append(df_test_3)

In [31]:
df_test_cc.shape

(842484, 20)

In [32]:
df_test_cc.to_csv('df_test_cc.csv',index=False)